# Car Detection - Model Training with YOLOv8

This notebook guides you through the process of training a YOLOv8 model for car detection using the dataset prepared with Roboflow.

## 1. Install Required Libraries

In [ ]:
!pip install ultralytics matplotlib tensorflow

## 2. Import Required Libraries

In [ ]:
import os
import yaml
from ultralytics import YOLO
import matplotlib.pyplot as plt
from IPython.display import Image, display

## 3. Define Dataset Path

Point to the dataset we prepared in the previous notebook.

In [ ]:
# Update this path to point to your dataset location
data_path = "../data/vehicles-detection-v1"

# Verify the data.yaml file exists
yaml_path = os.path.join(data_path, "data.yaml")
assert os.path.exists(yaml_path), f"data.yaml not found at {yaml_path}"

# Load and display dataset configuration
with open(yaml_path, "r") as f:
    data_config = yaml.safe_load(f)

print("Dataset configuration:")
for key, value in data_config.items():
    print(f"  {key}: {value}")

## 4. Initialize YOLO Model

We'll start with a pre-trained YOLOv8 model and fine-tune it for car detection.

In [ ]:
# Choose model size based on your requirements and computational resources
# Options: 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt'
model = YOLO('yolov8s.pt')  # We'll use the small model for training speed

print(f"Model loaded: {model.type}")

## 5. Model Training

Now we'll train the model on our car detection dataset. We'll define various hyperparameters for the training process.

In [ ]:
# Define training hyperparameters
# These can be adjusted based on your dataset and computational resources
epochs = 50
batch_size = 16
imgsz = 640  # Image size

# Start training
results = model.train(
    data=yaml_path,
    epochs=epochs,
    batch=batch_size,
    imgsz=imgsz,
    patience=10,  # Early stopping patience
    save=True,  # Save best model
    device='0',  # GPU device (use 'cpu' if no GPU available)
    project="../models",
    name="car_detection_model"
)

## 6. Evaluate the Model

After training, let's evaluate the model's performance on the validation set.

In [ ]:
# Path to the best trained model
best_model_path = "../models/car_detection_model/weights/best.pt"

# Load the model
trained_model = YOLO(best_model_path)

# Run validation
val_results = trained_model.val(data=yaml_path)

print(f"Validation mAP50: {val_results.box.map50:.4f}")
print(f"Validation mAP50-95: {val_results.box.map:.4f}")

## 7. Visualize Training Metrics

Let's visualize the training results to understand how our model improved over time.

In [ ]:
# Display training results plots
plots_path = os.path.join("../models/car_detection_model", "results.png")

if os.path.exists(plots_path):
    display(Image(filename=plots_path))
else:
    print(f"Training results plot not found at {plots_path}")

## 8. Run Inference on Sample Images

Let's test our trained model on some sample validation images.

In [ ]:
# Get a few sample images from the validation set
val_images_dir = os.path.join(data_path, "valid/images")
val_images = [os.path.join(val_images_dir, f) for f in os.listdir(val_images_dir)][:5]

# Run inference on sample images
for img_path in val_images:
    # Run prediction
    results = trained_model(img_path)
    
    # Get the result object
    result = results[0]
    
    # Plot results
    fig, ax = plt.subplots(figsize=(12, 9))
    ax.imshow(result.plot()[:, :, ::-1])  # Convert BGR to RGB
    plt.axis('off')
    plt.title(f"Predictions on {os.path.basename(img_path)}")
    plt.show()
    
    # Print detection counts
    boxes = result.boxes
    print(f"Image: {os.path.basename(img_path)}")
    print(f"  Detections: {len(boxes)}")
    
    if len(boxes) > 0:
        # Print confidence scores
        confidences = boxes.conf.tolist()
        classes = boxes.cls.tolist()
        class_names = [data_config['names'][int(c)] for c in classes]
        
        for i, (conf, cls_name) in enumerate(zip(confidences, class_names)):
            print(f"    {i+1}. {cls_name}: {conf:.2f}")

## 9. Apply Post-processing Image Techniques

Let's explore some post-processing techniques to improve detection visualization.

In [ ]:
import cv2
import numpy as np
import torch

def apply_post_processing(image_path, model, conf_threshold=0.25, iou_threshold=0.45):
    # Load image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Run prediction
    results = model(img_rgb, conf=conf_threshold, iou=iou_threshold)
    result = results[0]
    
    # Get detections
    boxes = result.boxes
    
    # Original detection
    original_plot = result.plot()
    original_plot_rgb = cv2.cvtColor(original_plot, cv2.COLOR_BGR2RGB)
    
    # Heat map visualization
    heatmap = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)
    
    if len(boxes) > 0:
        for box, conf in zip(boxes.xyxy, boxes.conf):
            x1, y1, x2, y2 = map(int, box)
            conf_value = float(conf)
            cv2.rectangle(heatmap, (x1, y1), (x2, y2), conf_value, -1)
    
    heatmap = cv2.normalize(heatmap, None, 0, 255, cv2.NORM_MINMAX)
    heatmap = np.uint8(heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    # Blend original image with heatmap
    heatmap_overlay = cv2.addWeighted(img, 0.7, heatmap, 0.3, 0)
    heatmap_overlay_rgb = cv2.cvtColor(heatmap_overlay, cv2.COLOR_BGR2RGB)
    
    # Visualization with blur background
    blurred_bg = cv2.GaussianBlur(img, (21, 21), 0)
    mask = np.zeros_like(img)
    
    if len(boxes) > 0:
        for box in boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            # Create mask for the detection region
            mask[y1:y2, x1:x2] = 255
    
    # Create focused image: blurred background with sharp detections
    focused_img = np.where(mask > 0, img, blurred_bg)
    focused_img_rgb = cv2.cvtColor(focused_img, cv2.COLOR_BGR2RGB)
    
    # Display results
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    axes[0, 0].imshow(img_rgb)
    axes[0, 0].set_title("Original Image")
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(original_plot_rgb)
    axes[0, 1].set_title("YOLO Detection")
    axes[0, 1].axis('off')
    
    axes[1, 0].imshow(heatmap_overlay_rgb)
    axes[1, 0].set_title("Confidence Heatmap Overlay")
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(focused_img_rgb)
    axes[1, 1].set_title("Focus Effect (Blur Background)")
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Apply post-processing to one sample image
if val_images:
    apply_post_processing(val_images[0], trained_model)

## 10. Export the Model for Inference

Finally, let's export our trained model in different formats for deployment.

In [ ]:
# Export the model to ONNX format
trained_model.export(format="onnx")

# Export to other formats if needed
# trained_model.export(format="tflite")
# trained_model.export(format="torchscript")

print("Model exported successfully!")

## 11. Save Model to Project Models Directory

In [ ]:
import shutil

# Source and destination paths
src_model_path = best_model_path
dst_model_path = "../models/best_car_detection.pt"

# Copy the best model
shutil.copy(src_model_path, dst_model_path)
print(f"Best model saved to {dst_model_path}")

## Next Steps

Now that you have trained a car detection model, you can use it for inference on new images and videos. See the `../src/detect.py` script for inference implementation.